#Data Import

In [ ]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("submission.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   store_id      int64  
 1   date          object 
 2   time          object 
 3   card_id       object 
 4   amount        int64  
 5   installments  float64
 6   days_of_week  int64  
 7   holyday       int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 205.2+ MB


In [4]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


#8등 코드

In [5]:
!pip install --upgrade pip
!pip install pyLDAvis
!pip install soyclustering==0.1.0

     |████████████████████████████████| 1.5MB 14.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.7 MB 17.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3 MB 258 kB/s 
     |████████████████████████████████| 9.9 MB 58.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136870 sha256=98ee31ec0b1f17b1561471ff9e57c8370fbf3f5c389bad94aaa6e6b44758bf03
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found ex

In [6]:
#위의 pip 들을 설치하면서 이 패키지들을 다시 import해야함
import pandas as pd
import numpy as np

In [7]:
import pyLDAvis
from sklearn.feature_extraction.text import CountVectorizer
from soyclustering import SphericalKMeans

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func


##Data preprocessing

###거래일자와 매출액을 활용한 상점 군집화

In [8]:
def convert_data(df, dateOption='str', amountOption=10):
    def convert_amount(df, amountOption=10):
        ##나누기 Option
        def divide_by_n(x, n):
            result = (x // n) * n
            if result == 0 :
                return "pos" + str(result)
            else :
                return "neg" + str(-result)
        df['amount_modified'] = df['amount'].apply(divide_by_n, args=(amountOption,))
        #print(df[['amount', 'amount_modified']].head())
        return df

    def convert_date(df, dateOption='str'):
        df['date'] = pd.to_datetime(df['date'])
        if dateOption == 'int':
            df['date_modified'] = df['date'].dt.dayofweek
        if dateOption == 'str':
            df['date_modified'] = df['date'].dt.day_name()

        #print(df[['date', 'date_modified']].head())
        return df
    
    df = convert_amount(df)
    df = convert_date(df)
    df["class"] = df["amount_modified"].map(str) +"_"+ df["date_modified"].map(str)
    
    return df

In [9]:
train = convert_data(train)

In [10]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday,amount_modified,date_modified,class
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0,pos0,Wednesday,pos0_Wednesday
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0,neg10,Wednesday,neg10_Wednesday
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0,neg-140,Monday,neg-140_Monday
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0,neg-60,Monday,neg-60_Monday
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0,neg-20,Monday,neg-20_Monday


In [11]:
print("거래의 총 Class의 갯수 : ", len(train['class'].unique()))

print("각 Store에 방문하는 거래 Class의 갯수 : ")
train.groupby('store_id')['class'].apply(lambda x: x.unique().shape[0])

거래의 총 Class의 갯수 :  6550
각 Store에 방문하는 거래 Class의 갯수 : 


store_id
0       789
1        30
2       443
3       140
4       259
       ... 
1795    447
1796     81
1797     33
1798    126
1799    132
Name: class, Length: 1775, dtype: int64

In [12]:
#상점별 Class를 띄어쓰기를 구분자로 문자열 LIST로 변환
new_train = train.groupby('store_id')['class'].apply(lambda x: "{%s}" % ' '.join(x))
strList = new_train.values
#vectorizing
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(strList)

In [13]:
spherical_kmeans = SphericalKMeans(
    n_clusters=100,
    max_iter=10,
    verbose=1,
    init='similar_cut'
)

labels = spherical_kmeans.fit_predict(X)
centers = spherical_kmeans.cluster_centers_

initialization_time=0.018342 sec, sparsity=0.0427
n_iter=1, changed=1730, inertia=29.696, iter_time=0.323 sec, sparsity=0.0911
n_iter=2, changed=474, inertia=24.825, iter_time=0.323 sec, sparsity=0.0904
n_iter=3, changed=262, inertia=24.204, iter_time=0.314 sec, sparsity=0.0885
n_iter=4, changed=162, inertia=24.006, iter_time=0.322 sec, sparsity=0.0871
n_iter=5, changed=118, inertia=23.954, iter_time=0.317 sec, sparsity=0.0868
n_iter=6, changed=97, inertia=23.916, iter_time=0.312 sec, sparsity=0.0866
n_iter=7, changed=48, inertia=23.894, iter_time=0.307 sec, sparsity=0.0861
n_iter=8, changed=28, inertia=23.886, iter_time=0.315 sec, sparsity=0.0859
n_iter=9, changed=27, inertia=23.887, iter_time=0.316 sec, sparsity=0.0858
n_iter=10, changed=17, inertia=23.882, iter_time=0.314 sec, sparsity=0.0858


In [15]:
!git clone https://github.com/lovit/kmeans_to_pyLDAvis
!pip install pyLDAvis

Cloning into 'kmeans_to_pyLDAvis'...
remote: Enumerating objects: 199, done.
remote: Total 199 (delta 0), reused 0 (delta 0), pack-reused 199
Receiving objects: 100% (199/199), 15.09 MiB | 17.80 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [16]:
import sys
sys.path.append('../')
import kmeans_to_pyLDAvis

#오류나는 부분
from kmeans_to_pyLDAvis import kmeans_to_prepared_data

vocab2int = vectorizer.vocabulary_
int2vocab = [
    word for word, index in sorted(
        vocab2int.items(), key=lambda x: x[1])
]
prepared_data = kmeans_to_prepared_data(
    X, int2vocab, centers, labels,
    embedding_method='tsne'
)

ImportError: ignored

In [ ]:
htmlFile = 'kmeans_pyLDAvis.html'
pyLDAvis.save_html(prepared_data, htmlFile)
pyLDAvis.display(prepared_data)

###군집화 한 정보를 시각화

In [ ]:
from bs4 import BeautifulSoup
# load the file
with open(htmlFile) as inf:
    txt = inf.read()
    soup = BeautifulSoup(txt)

In [ ]:
print(soup)
#tag.replace_with("선택한 그룹: ")